### This work is the result of my own tortures and talent of other author, whose ideas I borrowed (a bit). In order to be honest I include the link to his work in the end of the notebook.

###### When we plagiarize, we are likewise creating in the image and participating in the completion of Creation

                                                                             Jonathan Safran Foer, Everything Is Illuminated

In [239]:
import pandas as pd
import random
import nltk
nltk.download('stopwords')
from nltk import PorterStemmer
import string
from sklearn import naive_bayes
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Forest_Gump\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [240]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

### Let us import the data and look at them

In [241]:
#A-ha, 3 strange columns without names, which seem to contain only NaN
data = pd.read_csv('Downloads\smsspam.csv')
data.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [242]:
#These strange columns contain some info that can be useful further, so we cannot just drop them
data['Unnamed: 3'].unique()

array([nan, ' MK17 92H. 450Ppw 16"', ' why to miss them', 'GE',
       'U NO THECD ISV.IMPORTANT TOME 4 2MORO\\""',
       'i wil tolerat.bcs ur my someone..... But',
       ' ILLSPEAK 2 U2MORO WEN IM NOT ASLEEP...\\""',
       'whoever is the KING\\"!... Gud nyt"', ' TX 4 FONIN HON',
       ' \\"OH No! COMPETITION\\". Who knew', 'IеХL CALL U\\""'],
      dtype=object)

In [243]:
data.v1.value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

#### As measure of classification accuracy I decided to use 3 classic metrics: precision, recall and accuracy. 'Ham' is predicted as positive class, 'spam' - as negative. Therefore, we are interested in lower false negative (in order not to classify important ham messages as spam).If the classifier is good, I expect to see high recall for both ham and spam, and also high precision for spam. Accuracy in this case will give us general picture.

### For futher work with text we need to prepare messages

In [244]:
#defining a function for text preparation

def cleaning(cleantext):
    #lowercasing
    cleantext = [d.lower() for d in cleantext] 
    #removing punctuation and stopwords
    cleantext = ''.join([i for i in cleantext if i not in string.punctuation])
    cleantext = [i for i in cleantext.split() if i not in stopwords.words('english')]
    #stemming
    stem = PorterStemmer()
    cleantext = [stem.stem(i) for i in cleantext]
    return cleantext

In [245]:
#Applying cleaning function
data['v2'] = data['v2'].apply(cleaning)

In [341]:
#Let's check our clean data
data.head(2)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,predicted
0,ham,"[go, jurong, point, crazi, avail, bugi, n, gre...",NaN,NaN,NaN,ham
1,ham,"[ok, lar, joke, wif, u, oni]",NaN,NaN,NaN,ham


#### Some problem occured when I tried to perform 5-fold validation in a usual manner, therefore, I decided to make 5-fold validaion "by hands". I got indices through KFold and extracted sets from dataset with 'clean' messages. For each set a vocabulary was created, words were count, then through functions probabilities were calculated. Them classification was performed. Chosen metrics (precision, recall, accuracy) were also calculated "by hands". At the end of this strange cross-validation the 'table' with al counted metrics is presented.

In [247]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(data)
for train_index, test_index in kf.split(data):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = data['v2'][train_index], data['v2'][test_index]
    y_train, y_test = data['v1'][train_index], data['v1'][test_index]

TRAIN: [1115 1116 1117 ... 5569 5570 5571] TEST: [   0    1    2 ... 1112 1113 1114]
TRAIN: [   0    1    2 ... 5569 5570 5571] TEST: [1115 1116 1117 ... 2227 2228 2229]
TRAIN: [   0    1    2 ... 5569 5570 5571] TEST: [2230 2231 2232 ... 3341 3342 3343]
TRAIN: [   0    1    2 ... 5569 5570 5571] TEST: [3344 3345 3346 ... 4455 4456 4457]
TRAIN: [   0    1    2 ... 4455 4456 4457] TEST: [4458 4459 4460 ... 5569 5570 5571]


#### Extracting set 1 and applying classification fucntion 

In [248]:
#set1
X_train, X_test = data['v2'][1115:5572], data['v2'][0:1114]
y_train, y_test =  data['v1'][1115:5572],  data['v1'][0:1114]
train_data = pd.DataFrame([X_train, y_train])
test_data =pd.DataFrame([X_test, y_test])
train_data=train_data.transpose()
test_data=test_data.transpose()

In [249]:
vocabulary = list(set(train_data['v2'].sum()))
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [250]:
Pspam = train_data['v1'].value_counts()['spam'] / train_data.shape[0]
Pham = train_data['v1'].value_counts()['ham'] / train_data.shape[0]
Nspam = train_data.loc[train_data['v1'] == 'spam','v2'].apply(len).sum()
Nham = train_data.loc[train_data['v1'] == 'ham','v2'].apply(len).sum()
Nvoc = len(train_data.columns)
#setting lambda
l=1

In [251]:
def p_word_is_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'spam', word].sum() + l) / (Nspam + l*Nvoc)
    else:
        return 1
      
def p_word_is_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'ham', word].sum() + l) / (Nham + l*Nvoc)
    else:
        return 1

In [252]:
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_word_is_spam(word)
        p_ham_given_message *= p_word_is_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'

In [319]:
test_data['predicted']=test_data['v2'].apply(classify)

In [320]:
true=test_data['v1'].tolist()
predicted=test_data['predicted'].tolist()

In [321]:
true_positive1=0
true_negative1=0
false_positive1=0
false_negative1=0

for i in range(len(true)):
    if true[i]=='spam' and predicted[i]=='spam':
        true_negative1+=1
    elif true[i]=='spam' and predicted[i]=='ham':
        false_positive1+=1
    elif true[i]=='ham' and predicted[i]=='ham':
        true_positive1+=1
    elif true[i]=='ham' and predicted[i]=='spam':
        false_negative1+=1
    else:
        print('Error')

print('True positive 1: ', true_positive1)
print('True negative 1: ', true_negative1) 
print('False positive 1: ',false_positive1) 
print('False negative 1: ',false_negative1)

True positive 1:  961
True negative 1:  139
False positive 1:  6
False negative 1:  8


In [322]:
#Let us count the metrics: accuracy, precision and recall

hamprecision1=true_positive1/(true_positive1 + false_positive1)
hamrecall1=true_positive1/(true_positive1 + false_negative1)
spamprecision1=true_negative1/(true_negative1 + false_negative1)
spamrecall1=true_negative1/(true_negative1 + false_positive1)
accuracy1=(true_positive1+true_negative1)/(true_positive1+true_negative1+false_positive1+false_negative1)

print('Precision for ham 1: ',hamprecision1)
print('Recall for ham 1: ',hamrecall1)
print('Precision for spam 1: ',spamprecision1)
print('Recall for spam 1: ',spamrecall1)
print('Accuracy 1: ', accuracy1)

Precision for ham 1:  0.9937952430196484
Recall for ham 1:  0.9917440660474717
Precision for spam 1:  0.9455782312925171
Recall for spam 1:  0.9586206896551724
Accuracy 1:  0.9874326750448833


#### Extracting set 2 

In [257]:
#set2
X_train, X_test = data['v2'], data['v2'][1115:2229]
y_train, y_test =  data['v1'],  data['v1'][1115:2229]
train_data = pd.DataFrame([X_train, y_train])
test_data =pd.DataFrame([X_test, y_test])
train_data=train_data.transpose()
train_data = train_data.drop(train_data.index[1115:2229])
test_data=test_data.transpose()

In [258]:
vocabulary = list(set(train_data['v2'].sum()))
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [259]:
Pspam = train_data['v1'].value_counts()['spam'] / train_data.shape[0]
Pham = train_data['v1'].value_counts()['ham'] / train_data.shape[0]
Nspam = train_data.loc[train_data['v1'] == 'spam','v2'].apply(len).sum()
Nham = train_data.loc[train_data['v1'] == 'ham','v2'].apply(len).sum()
Nvoc = len(train_data.columns)
#setting lambda
l=1

In [260]:
def p_word_is_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'spam', word].sum() + l) / (Nspam + l*Nvoc)
    else:
        return 1
      
def p_word_is_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'ham', word].sum() + l) / (Nham + l*Nvoc)
    else:
        return 1

In [261]:
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_word_is_spam(word)
        p_ham_given_message *= p_word_is_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'

In [323]:
test_data['predicted']=test_data['v2'].apply(classify)

In [324]:
true=test_data['v1'].tolist()
predicted=test_data['predicted'].tolist()

In [325]:
true_positive2=0
true_negative2=0
false_positive2=0
false_negative2=0

for i in range(len(true)):
    if true[i]=='spam' and predicted[i]=='spam':
        true_negative2+=1
    elif true[i]=='spam' and predicted[i]=='ham':
        false_positive2+=1
    elif true[i]=='ham' and predicted[i]=='ham':
        true_positive2+=1
    elif true[i]=='ham' and predicted[i]=='spam':
        false_negative2+=1
    else:
        print('Error')

print('True positive 2: ', true_positive2)
print('True negative 2: ', true_negative2) 
print('False positive 2: ',false_positive2) 
print('False negative 2: ',false_negative2)

True positive 2:  961
True negative 2:  139
False positive 2:  6
False negative 2:  8


In [326]:
#Let us count the metrics: accuracy, precision and recall

hamprecision2=true_positive2/(true_positive2 + false_positive2)
hamrecall2=true_positive2/(true_positive2 + false_negative2)
spamprecision2=true_negative2/(true_negative2 + false_negative2)
spamrecall2=true_negative2/(true_negative2 + false_positive2)
accuracy2=(true_positive2+true_negative2)/(true_positive2+true_negative2+false_positive2+false_negative2)

print('Precision for ham 2: ',hamprecision2)
print('Recall for ham 2: ',hamrecall2)
print('Precision for spam 2: ',spamprecision2)
print('Recall for spam 2: ',spamrecall2)
print('Accuracy 2: ', accuracy2)

Precision for ham 2:  0.9937952430196484
Recall for ham 2:  0.9917440660474717
Precision for spam 2:  0.9455782312925171
Recall for spam 2:  0.9586206896551724
Accuracy 2:  0.9874326750448833


#### Extracting 3 set

In [266]:
#set3
X_train, X_test = data['v2'], data['v2'][2230:3344]
y_train, y_test =  data['v1'],  data['v1'][2230:3344]
train_data = pd.DataFrame([X_train, y_train])
test_data =pd.DataFrame([X_test, y_test])
train_data=train_data.transpose()
train_data = train_data.drop(train_data.index[2230:3344])
test_data=test_data.transpose()

In [267]:
vocabulary = list(set(train_data['v2'].sum()))
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [268]:
Pspam = train_data['v1'].value_counts()['spam'] / train_data.shape[0]
Pham = train_data['v1'].value_counts()['ham'] / train_data.shape[0]
Nspam = train_data.loc[train_data['v1'] == 'spam','v2'].apply(len).sum()
Nham = train_data.loc[train_data['v1'] == 'ham','v2'].apply(len).sum()
Nvoc = len(train_data.columns)
#setting lambda
l=1

In [269]:
def p_word_is_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'spam', word].sum() + l) / (Nspam + l*Nvoc)
    else:
        return 1
      
def p_word_is_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'ham', word].sum() + l) / (Nham + l*Nvoc)
    else:
        return 1

In [270]:
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_word_is_spam(word)
        p_ham_given_message *= p_word_is_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'

In [328]:
test_data['predicted']=test_data['v2'].apply(classify)

In [329]:
true=test_data['v1'].tolist()
predicted=test_data['predicted'].tolist()

In [330]:
true_positive3=0
true_negative3=0
false_positive3=0
false_negative3=0

for i in range(len(true)):
    if true[i]=='spam' and predicted[i]=='spam':
        true_negative3+=1
    elif true[i]=='spam' and predicted[i]=='ham':
        false_positive3+=1
    elif true[i]=='ham' and predicted[i]=='ham':
        true_positive3+=1
    elif true[i]=='ham' and predicted[i]=='spam':
        false_negative3+=1
    else:
        print('Error')

print('True positive 3: ', true_positive3)
print('True negative 3: ', true_negative3) 
print('False positive 3: ',false_positive3) 
print('False negative 3: ',false_negative3)

True positive 3:  961
True negative 3:  139
False positive 3:  6
False negative 3:  8


In [331]:
#Let us count the metrics: accuracy, precision and recall

hamprecision3=true_positive3/(true_positive3 + false_positive3)
hamrecall3=true_positive3/(true_positive3 + false_negative3)
spamprecision3=true_negative3/(true_negative3 + false_negative3)
spamrecall3=true_negative3/(true_negative3 + false_positive3)
accuracy3=(true_positive3+true_negative3)/(true_positive3+true_negative3+false_positive3+false_negative3)

print('Precision for ham 3: ',hamprecision3)
print('Recall for ham 3: ',hamrecall3)
print('Precision for spam 3: ',spamprecision3)
print('Recall for spam 3: ',spamrecall3)
print('Accuracy 3: ', accuracy3)

Precision for ham 3:  0.9937952430196484
Recall for ham 3:  0.9917440660474717
Precision for spam 3:  0.9455782312925171
Recall for spam 3:  0.9586206896551724
Accuracy 3:  0.9874326750448833


#### Extracting 4 set

In [275]:
#set4
X_train, X_test = data['v2'], data['v2'][3344:4458]
y_train, y_test =  data['v1'],  data['v1'][3344:4458]
train_data = pd.DataFrame([X_train, y_train])
test_data =pd.DataFrame([X_test, y_test])
train_data=train_data.transpose()
train_data = train_data.drop(train_data.index[3344:4458])
test_data=test_data.transpose()


In [276]:
vocabulary = list(set(train_data['v2'].sum()))
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [277]:
Pspam = train_data['v1'].value_counts()['spam'] / train_data.shape[0]
Pham = train_data['v1'].value_counts()['ham'] / train_data.shape[0]
Nspam = train_data.loc[train_data['v1'] == 'spam','v2'].apply(len).sum()
Nham = train_data.loc[train_data['v1'] == 'ham','v2'].apply(len).sum()
Nvoc = len(train_data.columns)
#setting lambda
l=1

In [278]:
def p_word_is_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'spam', word].sum() + l) / (Nspam + l*Nvoc)
    else:
        return 1
      
def p_word_is_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'ham', word].sum() + l) / (Nham + l*Nvoc)
    else:
        return 1

In [279]:
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_word_is_spam(word)
        p_ham_given_message *= p_word_is_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'

In [332]:
test_data['predicted']=test_data['v2'].apply(classify)

In [333]:
true=test_data['v1'].tolist()
predicted=test_data['predicted'].tolist()

In [334]:
true_positive4=0
true_negative4=0
false_positive4=0
false_negative4=0

for i in range(len(true)):
    if true[i]=='spam' and predicted[i]=='spam':
        true_negative4+=1
    elif true[i]=='spam' and predicted[i]=='ham':
        false_positive4+=1
    elif true[i]=='ham' and predicted[i]=='ham':
        true_positive4+=1
    elif true[i]=='ham' and predicted[i]=='spam':
        false_negative4+=1
    else:
        print('Error')

print('True positive 4: ', true_positive4)
print('True negative 4: ', true_negative4) 
print('False positive 4: ',false_positive4) 
print('False negative 4: ',false_negative4)

True positive 4:  961
True negative 4:  139
False positive 4:  6
False negative 4:  8


In [335]:
#Let us count the metrics: accuracy, precision and recall

hamprecision4=true_positive4/(true_positive4 + false_positive4)
hamrecall4=true_positive4/(true_positive4 + false_negative4)
spamprecision4=true_negative4/(true_negative4 + false_negative4)
spamrecall4=true_negative4/(true_negative4 + false_positive4)
accuracy4=(true_positive4+true_negative4)/(true_positive4+true_negative4+false_positive4+false_negative4)

print('Precision for ham 4: ',hamprecision4)
print('Recall for ham 4: ',hamrecall4)
print('Precision for spam 4: ',spamprecision4)
print('Recall for spam 4: ',spamrecall4)
print('Accuracy 4: ', accuracy4)

Precision for ham 4:  0.9937952430196484
Recall for ham 4:  0.9917440660474717
Precision for spam 4:  0.9455782312925171
Recall for spam 4:  0.9586206896551724
Accuracy 4:  0.9874326750448833


#### Extracting the last (5th) set

In [284]:
#set5
X_train, X_test = data['v2'], data['v2'][4458:5572]
y_train, y_test =  data['v1'],  data['v1'][4458:5572]
train_data = pd.DataFrame([X_train, y_train])
test_data =pd.DataFrame([X_test, y_test])
train_data=train_data.transpose()
train_data = train_data.drop(train_data.index[4458:5572])
test_data=test_data.transpose()

In [285]:
vocabulary = list(set(train_data['v2'].sum()))
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [286]:
Pspam = train_data['v1'].value_counts()['spam'] / train_data.shape[0]
Pham = train_data['v1'].value_counts()['ham'] / train_data.shape[0]
Nspam = train_data.loc[train_data['v1'] == 'spam','v2'].apply(len).sum()
Nham = train_data.loc[train_data['v1'] == 'ham','v2'].apply(len).sum()
Nvoc = len(train_data.columns)
#setting lambda
l=1

In [287]:
def p_word_is_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'spam', word].sum() + l) / (Nspam + l*Nvoc)
    else:
        return 1
      
def p_word_is_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'ham', word].sum() + l) / (Nham + l*Nvoc)
    else:
        return 1

In [288]:
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_word_is_spam(word)
        p_ham_given_message *= p_word_is_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'

In [344]:
test_data['predicted']=test_data['v2'].apply(classify)

In [337]:
true=test_data['v1'].tolist()
predicted=test_data['predicted'].tolist()

In [338]:
true_positive5=0
true_negative5=0
false_positive5=0
false_negative5=0

for i in range(len(true)):
    if true[i]=='spam' and predicted[i]=='spam':
        true_negative5+=1
    elif true[i]=='spam' and predicted[i]=='ham':
        false_positive5+=1
    elif true[i]=='ham' and predicted[i]=='ham':
        true_positive5+=1
    elif true[i]=='ham' and predicted[i]=='spam':
        false_negative5+=1
    else:
        print('Error')

print('True positive 5: ', true_positive5)
print('True negative 5: ', true_negative5) 
print('False positive 5: ',false_positive5) 
print('False negative 5: ',false_negative5)

True positive 5:  961
True negative 5:  139
False positive 5:  6
False negative 5:  8


In [339]:
#Let us count the metrics: accuracy, precision and recall

hamprecision5=true_positive5/(true_positive5 + false_positive5)
hamrecall5=true_positive5/(true_positive5 + false_negative5)
spamprecision5=true_negative5/(true_negative5 + false_negative5)
spamrecall5=true_negative5/(true_negative5 + false_positive5)
accuracy5=(true_positive5+true_negative5)/(true_positive5+true_negative5+false_positive5+false_negative5)

print('Precision for ham 5: ',hamprecision5)
print('Recall for ham 5: ',hamrecall5)
print('Precision for spam 5: ',spamprecision5)
print('Recall for spam 5: ',spamrecall5)
print('Accuracy 5: ', accuracy5)

Precision for ham 5:  0.9937952430196484
Recall for ham 5:  0.9917440660474717
Precision for spam 5:  0.9455782312925171
Recall for spam 5:  0.9586206896551724
Accuracy 5:  0.9874326750448833


In [342]:
metric={'Set №':['1', '2', '3', '4', '5'],
        'Ham precisions': [hamprecision1,hamprecision2,hamprecision2,hamprecision4,hamprecision5],
        'Spam precisions': [spamprecision1,spamprecision2,spamprecision2,spamprecision4,spamprecision5],
        'Ham recalls': [hamrecall1,hamrecall2,hamrecall3,hamrecall4,hamrecall5],
        'Spam recalls': [spamrecall1,spamrecall2,spamrecall3,spamrecall4,spamrecall5],
        'Accuracy':[accuracy1,accuracy2,accuracy3,accuracy4,accuracy5] }
metrics=pd.DataFrame(metric, columns = ['Set №', 'Ham precisions','Spam precisions', 'Ham recalls','Spam recalls','Accuracy'])
print(metrics)

  Set №  Ham precisions  Spam precisions  Ham recalls  Spam recalls  Accuracy
0     1        0.993795         0.945578     0.991744      0.958621  0.987433
1     2        0.993795         0.945578     0.991744      0.958621  0.987433
2     3        0.993795         0.945578     0.991744      0.958621  0.987433
3     4        0.993795         0.945578     0.991744      0.958621  0.987433
4     5        0.993795         0.945578     0.991744      0.958621  0.987433


#### As we can see, our target measures are quite high. For example, 95% of actual spam messages are classified as spam, and 99% of ham messages are classified correctly. Classification accuracy is equal to almost 99%, which is unexpectedly high for by hand coded classifier.

### Now it is time to use sklearn naive_bayes and see how it works

In [311]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report,confusion_matrix

In [312]:
newdata=pd.read_csv('Downloads\smsspam.csv')
newdata=newdata.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
newdata

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will М_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [313]:
X_train, X_test, y_train, y_test = train_test_split(newdata['v2'],newdata['v1'],test_size=0.2)

In [314]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer='word')), # converts strings to integer counts
    ('tfidf',TfidfTransformer()), # converts integer counts to weighted TF-IDF scores
    ('classifier',MultinomialNB()) # train on TF-IDF vectors with Naive Bayes classifier
])

In [315]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [316]:
predictions=pipeline.predict(X_test)

In [317]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.95      1.00      0.97       960
        spam       1.00      0.67      0.80       155

    accuracy                           0.95      1115
   macro avg       0.97      0.84      0.89      1115
weighted avg       0.96      0.95      0.95      1115



#### For sklearn naive_bayes results are a bit worse in terms of accuracy - 95% vs 99% got by 'hand-coded' classifier. 100% of ham messages are classified correctly, however recall for spam is much lower - only 67% of spam messages are classified as spam

### General conclusion

#### Coded 'by hand' classifier performs ham/spam classification better than inbuilt sklearn Multinomial NB classifier. All calculated measures of accuracy are higher for hand-coded NB (and this is a great surprise for me :) ). Accuracy for sklern NB is worse by 4%, and spam recall is lower by almost 30%. All in all, even though the 5-fold validation was made in awful manner, the hand-coded NB classifier shows better results. Hooray!

Link to Pavel Horbonos's work: https://towardsdatascience.com/how-to-build-and-apply-naive-bayes-classification-for-spam-filtering-2b8d3308501